In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.stats as st
from matplotlib.ticker import FuncFormatter
import scipy.optimize as op
from preprocessing import *
plt.style.use("dark_background") # Config plots for dark mode, delete if on light mode
plt.rcParams['figure.dpi'] = 150 # Hi-res plots

In [12]:
def N(t_scalar, t):
    """
    Returns the number of times in t less than or equal to t_scalar.
    Is used to compute N(t_{i,k}) and N'(t_{i,k}) depending on whether t above is t or t_prime
    """
    return np.searchsorted(t, t_scalar, side="right")

In [13]:
def m5_compensator_helper(t_scalar, t_prime, durations):
    """
    Compute the lists that are required for the compensator function but don't depend on the parameters
    """
    t_prime_difference = t_scalar - t_prime[:N(t_scalar, t_prime)]
    d_prime_1 = durations[:N(t_scalar, t_prime)] - 1
    d_prime_30 = np.absolute(durations[:N(t_scalar, t_prime)] - 30)

    return t_prime_difference, d_prime_1, d_prime_30

def m5_recursion_helper(t, t_prime, durations):
    """
    Compute the lists required for the recursive definitions of B_i and C_i
    """
    T = end_T
    time_differences = [] # Holds time differences t_ih - t'_ik
    d1_values = [] # Holds values d'_ik - 1
    d30_values = [] # Holds values |d'_ik - 30|

    for h in range(N(T, t)):
        # Construct list of t_ih - t'_ik for k = 1 to N'(T)
        differences_list = t[h] - t_prime[N(t[h-1], t_prime):N(t[h], t_prime)]

        # Construct list of d_ik - 1 for k = 1 to N'(T)
        d1_values_list = durations[N(t[h-1], t_prime):N(t[h], t_prime)] - 1

        # Construct list of |d_ik - 30| for k = 1 to N'(T)
        d30_values_list = np.absolute(durations[N(t[h-1], t_prime):N(t[h], t_prime)] - 30)

        time_differences.append(differences_list)
        d1_values.append(d1_values_list)
        d30_values.append(d30_values_list)


    return time_differences, d1_values, d30_values

In [14]:
def compensator_m5(t_scalar, compensator_helpers, lambda_i, alpha_i,
 beta_i, beta_i_prime, gamma_i, delta_i, delta_i_prime):
    """
    Evaluate compensator function for model 5.

    t_scalar: Scalar value at which we evaluate the compensator
    t_prime_precomputed: time differences (t_scalar - t_prime[:N(t_scalar, t_prime)]) array computed outside the function for efficiency
    durations: array of journey durations d'_{i,k} for each arrival time t'_{i,k} at station i
    The rest are parameters in model 5.   
    """
        
    t_prime_precomputed = compensator_helpers[0]
    d_prime_1 = compensator_helpers[1]
    d_prime_30 = compensator_helpers[2]


    term1 = lambda_i * t_scalar
    
    term2 = -(alpha_i / beta_i_prime) * np.sum(np.exp(-beta_i_prime * (t_prime_precomputed) - beta_i * d_prime_1)-1)

    term3 = -(gamma_i / delta_i_prime) * np.sum(np.exp(-delta_i_prime * (t_prime_precomputed) - delta_i * d_prime_30)-1)

    return term1 + term2 + term3

In [15]:
def B_i(h, t, t_prime, durations, beta_i, beta_i_prime, recursion_helpers):

    """
    Returns a list of [B_i(1), ..., B_i(h)] for model 5

    Note all index variables such as h, k, etc start at 1, like the mathematical notation.
    """

    B = [np.sum(np.exp(-beta_i * (durations[:N(t[0], t_prime)] - 1)) * np.exp(-beta_i_prime*(t[0] - t_prime[:N(t[0], t_prime)])))]

    time_differences = recursion_helpers[0]
    d1_values = recursion_helpers[1]
    #d30_values = recursion_helpers[2]

    # Append the rest
    for l in range(2, h+1):
        # First term in recursive formula for B_i(h)
        term1 = np.exp(-beta_i_prime*(t[l-1] - t[l-2])) * B[l-2]

        term2 = np.sum(np.exp(-beta_i * (d1_values[l-1])) * np.exp(-beta_i_prime*(time_differences[l-1])))
        B.append(term1 + term2)
    return np.array(B)
    

def C_i(h, t, t_prime, durations, delta_i, delta_i_prime, recursion_helpers):

    """
    Returns a list of [C_i(1), ..., C_i(h)] for model 5

    Note all index variables such as h, k, etc start at 1, like the mathematical notation.
    """

    C = [np.sum(np.exp(-delta_i * (durations[:N(t[0], t_prime)] - 1)) * np.exp(-delta_i_prime*(t[0] - t_prime[:N(t[0], t_prime)])))]

    time_differences = recursion_helpers[0]
    #d1_values = recursion_helpers[1]
    d30_values = recursion_helpers[2]

    # Append the rest
    for l in range(2, h+1):
        # First term in recursive formula for C_i(h)
        term1 = np.exp(-delta_i_prime*(t[l-1] - t[l-2])) * C[l-2]
        term2 = np.sum(np.exp(-delta_i * (d30_values[l-1])) * np.exp(-delta_i_prime*(time_differences[l-1])))
        C.append(term1 + term2)
    return np.array(C)

In [16]:
def m5_log_likelihood(t, t_prime, durations, compensator_helpers, recursion_helpers, lambda_i,
 alpha_i, beta_i, beta_i_prime, gamma_i, delta_i, delta_i_prime):
   
   T = end_T

   B_ = B_i(len(t), t, t_prime, durations, beta_i, beta_i_prime, recursion_helpers)

   C_ = C_i(len(t), t, t_prime, durations, delta_i, delta_i_prime, recursion_helpers)

   term1 = np.sum(np.log(lambda_i + alpha_i*B_ + delta_i*C_))

   term2 = -compensator_m5(T, compensator_helpers, lambda_i, alpha_i,
      beta_i, beta_i_prime, gamma_i, delta_i, delta_i_prime)

   return term1 + term2

### TODO: Make sure the durations d_ik correspond exactly to the end times t'_ik. Do we need to get durations from train times sorted by end_time?

In [17]:
durations_dict = {}
for st_id in train_sorted_start_ids:
    station_train_data = train_bike_data[train_bike_data.end_id == st_id]
    durations_dict[st_id] = station_train_data.duration.to_numpy()

In [20]:
# Test m5 likelihood function

t = t_per_station[1]
t_prime = t_prime_per_station[1]
durations = durations_dict[1]

recursion_helpers = m5_recursion_helper(t, t_prime, durations)
compensator_helpers = m5_compensator_helper(end_T, t_prime, durations)

print(m5_log_likelihood(t, t_prime, durations,
    compensator_helpers, recursion_helpers, 0.1, 0.01, 0.1, 0.1, 0.1, 0.01, 0.011))

-28288.909827850715
